In [2]:
import sys  
sys.path.insert(0, '..')

from config import Config

import pandas as pd
from sqlalchemy import create_engine

db = create_engine(Config.SQLALCHEMY_DATABASE_URI)

In [3]:
mista = pd.read_sql_query(
    """
    select ockovaci_mista.id ockovaci_misto_id, ockovaci_mista.nazev, okres_id, kraj_id 
    from ockovaci_mista join okresy on ockovaci_mista.okres_id=okresy.id;
    """,
    db
)
# mista

In [8]:
df = pd.read_sql_query("select * from ockovani_distribuce limit 10;", db)
# df

,datum,ockovaci_misto_id,cilove_ockovaci_misto_id,ockovaci_latka,vyrobce,akce,pocet_ampulek,pocet_davek
0,2020-12-26,77079458-38f2-4bff-b983-7f99a07afdc3,,Comirnaty,Pfizer,Příjem,390,2340
1,2020-12-26,98c76ec8-f7d1-474a-927c-d4a4fb2dcd0a,,Comirnaty,Pfizer,Příjem,390,2340
2,2020-12-26,9295cb5e-e9ae-4858-97ce-5dadaf99ad72,,Comirnaty,Pfizer,Příjem,195,1170
3,2020-12-27,4a444781-6d8f-496e-9da0-5f813f6113a6,9a2da984-2c54-4427-9cc2-6c63a4694299,Comirnaty,Pfizer,Výdej,26,156
4,2020-12-28,0f396137-e1fe-4e67-8291-292e4066376d,262721a0-5551-41d6-bc46-06d5aa9d18a8,Comirnaty,Pfizer,Výdej,140,840
5,2020-12-28,262721a0-5551-41d6-bc46-06d5aa9d18a8,262721a0-5551-41d6-bc46-06d5aa9d18a8,Comirnaty,Pfizer,Výdej,49,294
6,2020-12-29,57932100-0df4-472c-a015-d2b750df5b58,,Comirnaty,Pfizer,Příjem,195,1170
7,2020-12-30,e342972a-ccb3-4a9a-9a9f-ac95a11997eb,,Comirnaty,Pfizer,Příjem,195,1170
8,2020-12-30,ade1335c-0185-4292-af9a-5de58be97774,,Comirnaty,Pfizer,Příjem,195,1170
9,2020-12-30,4475c51f-69b3-410d-b404-eaf319711512,,Comirnaty,Pfizer,Příjem,195,1170


In [9]:
prijato = pd.read_sql_query(
    """
    select ockovaci_misto_id, vyrobce, sum(pocet_davek) prijato
    from ockovani_distribuce 
    where akce = 'Příjem' 
    group by (ockovaci_misto_id, vyrobce);
    """,
    db
)
# prijato

,ockovaci_misto_id,vyrobce,prijato
0,6ce2ad8f-b1f6-4252-b6f0-e8029219c17b,Moderna,1100
1,7fd8c9d3-96af-4379-be5c-63849cf5e013,Moderna,400
2,31de7f09-960c-4287-80b4-012568de1703,Moderna,1000
3,44133735-28b0-4c07-a675-f6613a761f8c,Moderna,5000
4,96d6fa7c-79b5-4f4e-9192-9ad7d1a332c7,Pfizer,25740
...,...,...,...
166,e8de7565-5dad-46a9-b7c1-7d7d7d73263f,AstraZeneca,400
167,fe82349e-63db-4623-b6c1-cabd2214c1ee,AstraZeneca,200
168,96d6fa7c-79b5-4f4e-9192-9ad7d1a332c7,AstraZeneca,21100
169,fe82349e-63db-4623-b6c1-cabd2214c1ee,Moderna,200


In [197]:
prijato_odjinud = pd.read_sql_query(
    """
    select cilove_ockovaci_misto_id ockovaci_misto_id, vyrobce, sum(pocet_davek) prijato_odjinud
    from ockovani_distribuce 
    where akce = 'Výdej' 
    group by (cilove_ockovaci_misto_id, vyrobce);
    """,
    db
)
# prijato_odjinud

In [198]:
vydano = pd.read_sql_query(
    """
    select ockovaci_misto_id, vyrobce, sum(pocet_davek) vydano
    from ockovani_distribuce 
    where akce = 'Výdej' 
    group by (ockovaci_misto_id, vyrobce);
    """,
    db
)
# vydano

In [199]:
df = pd.read_sql_query("select * from ockovani_spotreba limit 10;", db)
# df

In [200]:
spotreba = pd.read_sql_query(
    """
    select ockovaci_misto_id, vyrobce, sum(pouzite_davky) pouzito, sum(znehodnocene_davky) znehodnoceno
    from ockovani_spotreba 
    group by (ockovaci_misto_id, vyrobce);
    """,
    db
)
# spotreba

In [238]:
mista_key = mista
mista_key['join'] = 0

vyrobci_key = spotreba[['vyrobce']].drop_duplicates()
vyrobci_key['join'] = 0

merged = mista_key.merge(vyrobci_key).drop('join', axis=1)

merged = pd.merge(merged, prijato, how="left")
merged = pd.merge(merged, prijato_odjinud, how="left")
merged = pd.merge(merged, vydano, how="left")
merged = pd.merge(merged, spotreba, how="left")

merged['prijato'] = merged['prijato'].fillna(0).astype('int')
merged['prijato_odjinud'] = merged['prijato_odjinud'].fillna(0).astype('int')
merged['vydano'] = merged['vydano'].fillna(0).astype('int')
merged['pouzito'] = merged['pouzito'].fillna(0).astype('int')
merged['znehodnoceno'] = merged['znehodnoceno'].fillna(0).astype('int')
merged['skladem'] = merged['prijato'] + merged['prijato_odjinud'] - merged['vydano'] - merged['pouzito'] - merged['znehodnoceno']

merged

,ockovaci_misto_id,nazev,okres_id,kraj_id,vyrobce,prijato,prijato_odjinud,vydano,pouzito,znehodnoceno,skladem
0,63c9078b-0701-4da0-821e-4eaa0d9c9968,"Krajská zdravotní a.s., - MNUL pracoviště výst...",CZ0423,CZ042,Moderna,0,0,0,0,0,0
1,63c9078b-0701-4da0-821e-4eaa0d9c9968,"Krajská zdravotní a.s., - MNUL pracoviště výst...",CZ0423,CZ042,Pfizer,0,0,0,0,0,0
2,63c9078b-0701-4da0-821e-4eaa0d9c9968,"Krajská zdravotní a.s., - MNUL pracoviště výst...",CZ0423,CZ042,AstraZeneca,0,0,0,0,0,0
3,989b3700-401f-4700-a684-ae09d348a3c0,Nemocnice Hořovice,CZ0202,CZ020,Moderna,900,0,70,530,0,300
4,989b3700-401f-4700-a684-ae09d348a3c0,Nemocnice Hořovice,CZ0202,CZ020,Pfizer,0,4068,150,3066,30,822
...,...,...,...,...,...,...,...,...,...,...,...
829,98c76ec8-f7d1-474a-927c-d4a4fb2dcd0a,ÚVN Praha - sklad,CZ0100,CZ010,Pfizer,32760,2340,22518,8694,18,3870
830,98c76ec8-f7d1-474a-927c-d4a4fb2dcd0a,ÚVN Praha - sklad,CZ0100,CZ010,AstraZeneca,0,0,0,0,0,0
831,cb0f8a24-3c54-467e-9e67-de01f424ea85,Česká Lípa očkovací místo Crystal,CZ0511,CZ051,Moderna,0,0,0,0,0,0
832,cb0f8a24-3c54-467e-9e67-de01f424ea85,Česká Lípa očkovací místo Crystal,CZ0511,CZ051,Pfizer,0,2874,0,3114,0,-240
